# PubMed Knowledge Graph

This notebook is part of a series that walks through the process of generating a knowledge graph of PubMed articles.

This notebook will
* Load structured patient journey data into a Neo4j instance

In [1]:
import os

import pandas as pd
from pyneoinstance import Neo4jInstance, load_yaml_file

## PyNeoInstance

Our database credentials and all of our queries are stored in the `pyneoinstance_config.yaml` file. 

This makes it easy to manage our queries and keeps the notebook code clean. 

In [2]:
config = load_yaml_file("pyneoinstance_config.yaml")

db_info = config['db_info']

constraints = config['initializing_queries']['constraints']
indexes = config['initializing_queries']['indexes']

claim_queries = config['loading_queries']['patient_journey_claims']
protocol_query = config['loading_queries']['patient_journey_protocol']
post_processing_queries = config['processing_queries']

This graph object will handle database connections and read / write transactions for us.

In [3]:
graph = Neo4jInstance(db_info.get('uri', os.getenv("NEO4J_URI", "neo4j://localhost:7687")), # use config value -> use env value -> use default value
                      db_info.get('user', os.getenv("NEO4J_USER", "neo4j")), 
                      db_info.get('password', os.getenv("NEO4J_PASSWORD", "password")))

This is a helper function for ingesting data using the PyNeoInstance library.

In [4]:
def get_partition(data: pd.DataFrame, batch_size: int = 500) -> int:
    """
    Determine the data partition based on the desired batch size.

    Parameters
    ----------
    data : pd.DataFrame
        The Pandas DataFrame to partition.
    batch_size : int
        The desired batch size.

    Returns
    -------
    int
        The partition size.
    """
    
    partition = int(len(data) / batch_size)
    print("partition: "+str(partition if partition > 1 else 1))
    return partition if partition > 1 else 1

In [5]:
icd9_to_icd10 = {
    '244.9': 'E03.9',      # Hypothyroidism, unspecified
    '250.0': 'E11.9',      # Diabetes mellitus type 2 without complications
    '272.4': 'E78.5',      # Hyperlipidemia, unspecified
    '300.0': 'F41.9',      # Anxiety disorder, unspecified
    '311.0': 'F32.9',      # Major depressive disorder, single episode, unspecified
    '401.9': 'I10',        # Essential (primary) hypertension
    '414.0': 'I25.10',     # Atherosclerotic heart disease without angina pectoris
    '493.9': 'J45.909',    # Asthma, unspecified, uncomplicated
    '496.0': 'J44.9',      # COPD, unspecified
    '530.81': 'K21.9',     # Gastroesophageal reflux disease without esophagitis
    '585.9': 'N18.9',      # Chronic kidney disease, unspecified
    '715.9': 'M19.90'      # Osteoarthritis, unspecified site
}

In [6]:
data = pd.read_csv("data/claims/claims_with_all_codes.csv", converters={'icd9': str})

In [7]:
data["icd10"] = data["icd9"].map(icd9_to_icd10)

In [8]:
data.head()

,claim_id,patient_id,date,procedure,amount,cpt4,icd9,provider_id,rxnorm_code,ndc_code,icd10
0,RX28558,P001,2024-01-06,Prescription: Losartan,28.30,J3490,244.9,PR004,861008.0,00006-0951-54,E03.9
1,RX70886,P001,2024-10-28,Prescription: Atorvastatin,37.82,J3490,414.0,PR012,617314.0,00078-0486-15,I25.10
2,C82031,P001,2024-01-06,Specialist Visit,669.26,99243,530.81,PR011,NaN,NaN,K21.9
3,C86208,P001,2024-12-28,HbA1c Test,148.37,83036,530.81,PR012,NaN,NaN,K21.9
4,C67045,P001,2025-01-20,HbA1c Test,1332.01,83036,300.0,PR004,NaN,NaN,F41.9


## Constraints + Indexes

In [9]:
def create_constraints_and_indexes() -> None:
    """
    Create constraints and indexes for the lexical and domain graphs.
    """
    try:
        if constraints and len(constraints) > 0:
            graph.execute_write_queries(database=db_info['database'], queries=list(constraints.values()))
    except Exception as e:
        print(e)

    try:
        if indexes and len(indexes) > 0:
            graph.execute_write_queries(database=db_info['database'], queries=list(indexes.values()))
    except Exception as e:
        print(e)

In [10]:
create_constraints_and_indexes()

## Load Data

Our patient journey data will be loaded from a Pandas DataFrame.

In [11]:
def load_patient_journey_protocol_data(graph: Neo4jInstance, data: pd.DataFrame) -> None:
    """
    Load patient journey protocol data into the graph.

    Parameters
    ----------
    graph: Neo4jInstance
        The graph to load the data into.
    dataframe_tuples: list[tuple[str, pd.DataFrame]]
        A list of tuples, where the first element is the name of the query in the config and the second element is the associated dataframe.
    """
    print(f"Loading {len(data)} patient journey protocolrows")
    
    res = graph.execute_write_query_with_data(database=db_info['database'], 
                                            data=data, 
                                            query=protocol_query,
                                            partitions=get_partition(data, batch_size=500),
                                            parallel=False)
    print(res)
def load_patient_journey_claims_data(graph: Neo4jInstance, dataframe_tuples: list[tuple[str, pd.DataFrame]]) -> None:
    """
    Load patient journey claimsdata into the graph.

    Parameters
    ----------
    graph: Neo4jInstance
        The graph to load the data into.
    dataframe_tuples: list[tuple[str, pd.DataFrame]]
        A list of tuples, where the first element is the name of the query in the config and the second element is the associated dataframe.
    """

    # load csv data
    for query_name, dataframe in dataframe_tuples:
        print(f"Loading {len(dataframe)} {query_name} rows")
        res = graph.execute_write_query_with_data(database=db_info['database'], 
                                                data=dataframe, 
                                                query=claim_queries[query_name], 
                                                partitions=get_partition(dataframe, batch_size=500),
                                                parallel=False)
        print(res)

def sequence_patient_events(graph: Neo4jInstance) -> None:
    """
    Post processing function to connect all patient events sequentially.
    Patient will be connected to their most recent event.

    (:Event)<-[:PREVIOUS]-(:Event)<-[:MOST_RECENT_EVENT]->(:Patient)
    """
    res = graph.execute_write_query(database=db_info['database'], 
                                    query=post_processing_queries['sequence_patient_events'])
    print(res)

In [12]:
protocol_data = pd.read_csv("data/protocol/extended_patient_journey.csv")

In [13]:
claims_data_path = "data/claims"
claims_data_tuples = [
    ("patients", pd.read_csv(f"{claims_data_path}/patients.csv")),
    ("providers", pd.read_csv(f"{claims_data_path}/providers.csv")),
    ("claims", pd.read_csv(f"{claims_data_path}/claims_with_all_codes.csv", converters={'icd9': str})),
    ("events", pd.read_csv(f"{claims_data_path}/patient_journey_with_providers.csv")),
    ("conditions", pd.read_csv(f"{claims_data_path}/conditions.csv")),
    ("care_gaps", pd.read_csv(f"{claims_data_path}/care_gaps.csv")),
    ("risk_scores", pd.read_csv(f"{claims_data_path}/risk_scores.csv"))
]

And now we load our patient journey data

In [14]:
load_patient_journey_protocol_data(graph, protocol_data)

Loading 221 patient journey protocolrows
partition: 1
{'labels_added': 139, 'relationships_created': 863, 'nodes_created': 139, 'properties_set': 938}


In [15]:
load_patient_journey_claims_data(graph, claims_data_tuples)

Loading 50 patients rows
partition: 1
{'properties_set': 100}
Loading 20 providers rows
partition: 1
{'labels_added': 20, 'nodes_created': 20, 'properties_set': 60}
Loading 310 claims rows
partition: 1
{'labels_added': 617, 'relationships_created': 620, 'nodes_created': 617, 'properties_set': 2784}
Loading 333 events rows
partition: 1
{'labels_added': 329, 'relationships_created': 333, 'nodes_created': 329, 'properties_set': 991}
Loading 86 conditions rows
partition: 1
{'labels_added': 12, 'relationships_created': 86, 'nodes_created': 12, 'properties_set': 12}
Loading 43 care_gaps rows
partition: 1
{'labels_added': 50, 'relationships_created': 86, 'nodes_created': 50, 'properties_set': 136}
Loading 50 risk_scores rows
partition: 1
{'labels_added': 1, 'relationships_created': 50, 'nodes_created': 1, 'properties_set': 100}


In [16]:
sequence_patient_events(graph)

{'relationships_created': 376, 'relationships_deleted': 376}
